In [2]:
import soundfile as sf
import math
import numpy as np
import librosa as lb
from IPython.display import Audio
from librosa import feature, frames_to_time, autocorrelate, midi_to_hz
import pandas as pd


In [63]:
x, fsx = lb.load('../uploaded_audio/Radiohead_track3.wav', sr=22050)

In [64]:
def getKey(audio, nfft=4096, fs=22050):
    # this function gets the most likely key of an array by taking a chromagram of the values and finding 
    #   the major or minor key to which it is most correlated
    
    # Parameters:
    #   audio- array_like
    #          array of values to be analyzed
    #   nfft- int, optional
    #         Size for Fast Fourier Transform in the chromagram function from librosa.Defaults to 4096 (large fft size)
    #   fs-   int, optional
    #         sample rate. Defaults to 22050  
    import operator
    normalized = audio / abs(np.max(audio)) # normalize audio file
    
    chromagram = feature.chroma_stft(normalized, n_fft = nfft, hop_length = int(nfft/4), sr = fs) # makes a chromagram 
    chromagram = chromagram.mean(axis=1) # averages the chromagram
    
    major_0 = np.array([6.35,2.23,3.48,2.33,4.38,4.09,2.52,5.19,2.39,3.66,2.29,2.88]) #C Major #values: C,C#,D,D#,...B
    minor_0 = np.array([6.33,2.68,3.52,5.38,2.60,3.53,2.54,4.75,3.98,2.69,3.34,3.17]) #C Minor #values: C,C#,D,D#,...B
    
    arraysDict = {}
    for i in range(0,12):
        arraysDict['major_{0}'.format(i)] = np.roll(major_0,i)# iteratively create each label while rotating for both
        arraysDict['minor_{0}'.format(i)] = np.roll(minor_0,i)#   major and minor keys
    
    # Dictionary below takes arrayDict and assigns the key type/number a note name
    keyTypeToName = {'major_0' : 'C', 'minor_0' : 'c', 'major_1' : 'C#/Db', 'minor_1' : 'c#/db'
    , 'major_2' : 'D', 'minor_2' : 'd', 'major_3' : 'D#/Eb', 'minor_3' : 'd#/eb', 'major_4' : 'E', 'minor_4' : 'e'
    , 'major_5' : 'F', 'minor_5' : 'f', 'major_6' : 'F#/Gb', 'minor_6' : 'f#/gb', 'major_7' : 'G', 'minor_7' : 'g'
    , 'major_8' : 'G#/Ab', 'minor_8' : 'g#/ab', 'major_9' : 'A', 'minor_9' : 'a', 'major_10' : 'A#/Bb'
    , 'minor_10' : 'a#/bb', 'major_11' : 'B', 'minor_11' : 'b'}
    
    finalDict = {} # dictionary of possible key profiles
    for key, value in arraysDict.items():
        relate = np.corrcoef([chromagram,value]) # compare chromagram to Krumhansl and Kessler key profiles
        finalDict[key] = relate[0][1] # add correletion coefficient to dictionary of possible key profiles
    
    maxChecker = max(finalDict.items(), key = operator.itemgetter(1)) # find maximum correlation coefficient
    guessKeyType = maxChecker[0] # get maximum correlation coefficient's key type/number
    guessKeyName = keyTypeToName[guessKeyType] # convert key type/number to key name using keyTypeToName dictionary
    return guessKeyName # return most correlated key                    
getKey(x, fs = fsx)

'c'

In [51]:
def getKeysFromFiles(directory, nfft=4096, fs=22050, percentOfSong = .25):
    # this function iterates through a directory of files and gets the most correlated key
    # Parameters:
    #   audio- array_like
    #          array of values to be analyzed
    #   nfft- int, optional
    #         Size for Fast Fourier Transform in the chromagram function from librosa.Defaults to 4096 (large fft size)
    #   fs-   int, optional
    #         sample rate. Defaults to 22050 
    #   percentOfSong- float, optional
    #         amount of each file to analyze in percentage. Defaults to 25% to save time
    import os
    dictOfKeys = {}
    for root, dirs, files in os.walk(directory): # get files from directory
        n = 0
        for file in files: # iterate through each file in directory of files
            try:
                x, fsx = lb.load(directory + '/' + file, sr = fs) # load file
                key = getKey(x[:int(x.size * percentOfSong)], nfft = nfft, fs = fsx) # get key using function from 
                dictOfKeys[file] = key # add key and filename to dictionary of keys   #  above
                n+=1
            except Exception as error: # handle any error that may come from reading the file
                print(error) # print Exception to notify user of error
            print(n) # print file number to track progress
    return dictOfKeys # returns dictionary with filenames as keys and key type/name as items
keys = getKeysFromFiles('../Keyfinding/KeyFinding_Audio/Audio') # run the function

# Runtime: 15 minutes

1
2
3
4
5
6
7


/Users/davidjones/AudioTech2/at2venv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


/Users/davidjones/AudioTech2/at2venv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


...

307
308


/Users/davidjones/AudioTech2/at2venv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


309
310
311
312
313
314
315
316
317
318


/Users/davidjones/AudioTech2/at2venv/lib/python3.9/site-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


319
320
321
322
323
324
325
326
327
328
329
330


In [50]:
guessedKey = pd.DataFrame(keys.items()) # create dataframe using dictionary from function above
guessedKey = guessedKey.sort_values(0, ignore_index = True) # sort values and reindex
guessedKey

,0,1
0,.DS_Store,NaN
1,0003_C.wav,a
2,0004_Ab.wav,A
3,0006_C.wav,C
4,0015_Eb.wav,D#/Eb
...,...,...
326,reggae.00095.au,C
327,reggae.00096.au,f
328,reggae.00097.au,a
329,reggae.00098.au,c


In [54]:
base = pd.read_csv('../Keyfinding/keys_corrected.csv') # read csv to compare base key to guessed key
base = base.sort_values('Title', ignore_index = True) # sort values and reindex
base

,Title,Composer,Date,Type,Unnamed: 4,Key,Source,Genre,SubGenre,Instrumentation,.MID,mid to wav?,Audio
0,0003_C.wav,NaN,NaN,NaN,NaN,C,Billboard,Popular,NaN,NaN,False,NaN,.wav
1,0004_Ab.wav,NaN,NaN,NaN,NaN,Ab,Billboard,Popular,NaN,NaN,False,NaN,.wav
2,0006_C.wav,NaN,NaN,NaN,NaN,C,Billboard,Popular,NaN,NaN,False,NaN,.wav
3,0015_Eb.wav,NaN,NaN,NaN,NaN,Eb,Billboard,Popular,NaN,NaN,False,NaN,.wav
4,0016_a.wav,NaN,NaN,NaN,NaN,a,Billboard,Popular,NaN,NaN,False,NaN,.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,reggae.00095.li.txt,NaN,NaN,NaN,NaN,C,GTZAN,Popular,NaN,NaN,NaN,NaN,NaN
326,reggae.00096.li.txt,NaN,NaN,NaN,NaN,C,GTZAN,Popular,NaN,NaN,NaN,NaN,NaN
327,reggae.00097.li.txt,NaN,NaN,NaN,NaN,F,GTZAN,Popular,NaN,NaN,NaN,NaN,NaN
328,reggae.00098.li.txt,NaN,NaN,NaN,NaN,c,GTZAN,Popular,NaN,NaN,NaN,NaN,NaN


In [55]:
compare = pd.DataFrame() # make dataframe
compare['File name'] = base['Title'] # take filenames from base case csv file and make a column
compare['Guessed File Name'] = guessedKey[0]
compare['Guessed Key'] = guessedKey[1] # make column of guessed keys
compare['Base Key'] = base['Key'] # make column of base keys
compare

,File name,Guessed File Name,Guessed Key,Base Key
0,0003_C.wav,.DS_Store,NaN,C
1,0004_Ab.wav,0003_C.wav,a,Ab
2,0006_C.wav,0004_Ab.wav,A,C
3,0015_Eb.wav,0006_C.wav,C,Eb
4,0016_a.wav,0015_Eb.wav,D#/Eb,a
...,...,...,...,...
325,reggae.00095.li.txt,reggae.00094.au,D#/Eb,C
326,reggae.00096.li.txt,reggae.00095.au,C,C
327,reggae.00097.li.txt,reggae.00096.au,f,F
328,reggae.00098.li.txt,reggae.00097.au,a,c


## Basic Analysis ##

In [56]:
# gives simple binary of whether the correct key or correct tonic were guessed
comparison_column = np.array([])
correct_tonic = np.array([])
for i, j in zip(compare['Base Key'], compare['Guessed Key']):
    if i in j:
        comparison_column = np.append(comparison_column, True) # if key is exact same, append true to comparison array
    else:
        comparison_column = np.append(comparison_column, False) # if not the exact same, append false
    if i.lower() in j.lower():
        correct_tonic = np.append(correct_tonic, True) # if correct tonic, append true to comparison array
    else:
        correct_tonic = np.append(correct_tonic, False) # if completely different append false

compare['Correct Key'] = comparison_column
compare['Correct Tonic'] = correct_tonic
compare

,File name,Guessed File Name,Guessed Key,Base Key,Correct Key,Correct Tonic
0,0003_C.wav,.DS_Store,NaN,C,0.0,0.0
1,0004_Ab.wav,0003_C.wav,a,Ab,0.0,0.0
2,0006_C.wav,0004_Ab.wav,A,C,0.0,0.0
3,0015_Eb.wav,0006_C.wav,C,Eb,0.0,0.0
4,0016_a.wav,0015_Eb.wav,D#/Eb,a,0.0,0.0
...,...,...,...,...,...,...
325,reggae.00095.li.txt,reggae.00094.au,D#/Eb,C,0.0,0.0
326,reggae.00096.li.txt,reggae.00095.au,C,C,1.0,1.0
327,reggae.00097.li.txt,reggae.00096.au,f,F,0.0,1.0
328,reggae.00098.li.txt,reggae.00097.au,a,c,0.0,0.0


In [57]:
counts = compare['Correct Key'].value_counts(), compare['Correct Tonic'].value_counts()
pd.DataFrame(counts)

,0.0,1.0
Correct Key,267,63
Correct Tonic,242,88


## In Depth Analysis ##

In [59]:
# score based on difference in key signatures and correct tonic
# ----
# correct key with tonic = 1pt
# relative major/minor = 0.8pt
# key signature 5th away = 0.66pt 
# correct tonic only = 0.5pt

# dictionary below assigns a number to a Key based on its position in the circle of fifths
letterToCircFifths = {
    "C":0, "a":0,
    "G":1, "e":1,
    "D":2, "b":2,
    "A":3, "f#/gb":3, "f#":3, "gb":3,
    "E":4, "c#/db":4, "c#":4, "db":4,
    "B":5, "g#/ab":5, "g#":5, "ab":5,
    "F#/Gb":6, "d#/eb":6, "F#":6, "Gb":6, "d#":6, "eb":6,
    "C#/Db":7, "a#/bb":7, "C#":7, "Db":7, "a#":7, "bb":7,
    "G#/Ab":8, "f":8, "G#":8, "Ab":8,
    "D#/Eb":9, "c":9, "D#":9, "Eb":9,
    "A#/Bb":10, "g":10, "A#":10, "Bb":10,
    "F":11, "d":11
}
# Circle of Fifths = CoF
keysig_score = np.array([]) # array of key scores
for i, j in zip(compare['Base Key'], compare['Guessed Key']): # iterate through guessed keys and base keys
    temp = abs(letterToCircFifths[i] - letterToCircFifths[j]) # find difference between guessed and base keys on CoF
    diff = min(temp, 12-temp) # gets minimum difference in both directions on CoF                                
    score = 0                  
    if diff == 0: # if CoF position is the same
        if i.lower() in j.lower():
            score = 1 # if same exact key, score = 1
        else:
            score = 0.8 # if relative major/minor, score = .8
    elif diff == 1:
        score = 0.66 # if difference in accidentals is only 1, 
    elif i.lower() in j.lower():
        score = 0.5 # if correct tonic but wrong major/minor quality, score = .5
    # every other difference number is given a key signature score of 0
    keysig_score = np.append(keysig_score, score)

compare['Key Signature Score'] = keysig_score
compare

KeyError: 'NaN'

In [41]:
print("Key Signature Score Accuracy:", str(np.sum(keysig_score)/keysig_score.size)) # print average key score accuracy
keySigScoreCounts = compare['Key Signature Score'].value_counts() # counts of each key score value
keySigScoreStats = pd.DataFrame()
keySigScoreStats['Counts'] = keySigScoreCounts
# line below gets percent of occurence of each key score value
keySigScoreStats["% of total"] = (100*keySigScoreStats['Counts']/keySigScoreStats['Counts'].sum()).round(decimals=1)
keySigScoreStats.sort_index(ascending=False)


Key Signature Score Accuracy: 0.5325454545454545


,Counts,% of total
1.00,96,29.1
0.80,12,3.6
0.66,79,23.9
0.50,36,10.9
0.00,107,32.4


In [ ]:
x, fsx = lb.load('../Keyfinding/KeyFinding_Audio/Audio/', sr=22050)